<a href="https://colab.research.google.com/github/rahuljungbahadur/hands_on_ml_book/blob/main/chp12_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises From chapter 12

## Do you get the same result with tf.range(10) and tf.constant(np.arange(10))?

In [3]:
import tensorflow as tf
import numpy as np

In [2]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [4]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

The later one has a default data type of `int64` (higher precision). Whereas the tensorflow one has a dtype of `int32` (lower precision)

## Sample python function converted to tensorlfow

In [5]:
@tf.function
def sum_squares(n):
  s=0
  for i in range(n):
    s+=i**2
  
  return s


In [7]:
sum_squares  ## It is a python funcition

In [9]:
tf.autograph.to_code(sum_squares.python_function)

"def tf__sum_squares(n):\n    with ag__.FunctionScope('sum_squares', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:\n        do_return = False\n        retval_ = ag__.UndefinedReturnValue()\n        s = 0\n\n        def get_state():\n            return (s,)\n\n        def set_state(vars_):\n            nonlocal s\n            (s,) = vars_\n\n        def loop_body(itr):\n            nonlocal s\n            i = itr\n            s = ag__.ld(s)\n            s += (i ** 2)\n        i = ag__.Undefined('i')\n        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(n),), None, fscope), None, loop_body, get_state, set_state, ('s',), {'iterate_names': 'i'})\n        try:\n            do_return = True\n            retval_ = ag__.ld(s)\n        except:\n            do_return = False\n            raise\n        return fscope.ret(retval_, do_return)\n"

In [ ]:
def tf__sum_squares(n):
    with ag__.FunctionScope('sum_squares', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state():
            return (s,)

        def set_state(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            i = itr
            s = ag__.ld(s)
            s += (i ** 2)
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(n),), None, fscope), None, loop_body, get_state, set_state, ('s',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(s)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)


This way you can see the tensorflow generated code

## Implement a custom layer that performs Layer Normalization (we will use this type of layer in Chapter 15):